# ECE 580 Project Code
## Fake News Detection using SVM and TF-IDF
- Rebecca Du (rrd17)
- Anish Parmar (avp30)

# Imports

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [4]:
#Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rebec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 1: Data Preprocessing

## Overview:

We are using the [ISOT Fake News Kaggle Dataset (2016-2017)](https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset). It is a dataset formed by news articles various sources. 

It is composed of 2 CSV files:
- **'Fake.csv'**: 23,481 articles from various unreliable websites
- **'True.csv'**: 21,417 articles from Reuters.com

Each CSV file contains the following features:
- **Title**: title of the 
- **Text**: The full body of text in the article
- **Subject**: News, Politics, Other
- **Date**: Date of publication in 'Month Day, Year' format

## Step 1a: Load, Label, Combine Datasets

Since the Fake and True datasets are provided separately, we first **assign them labels and concatenate them into one dataframe**, which will be randomized and later used for training and testing our model. 

In [8]:
#Load datasets as dataframes
fake_df = pd.read_csv('Data/Fake.csv')
true_df = pd.read_csv('Data/True.csv')

In [9]:
#Check fake_df
fake_df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [10]:
#Check true_df
true_df

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [11]:
#Label fake_df values as '0', true_df values as '1'
fake_df['label'] = 0
true_df['label'] = 1

#Combine datasets into one
data = pd.concat([fake_df, true_df], ignore_index=True)

#Check data
data

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


## Step 1b: Combine Relevant Columns into 'Content'

Since we are mostly focused on the content of the articles, we will combine the title with the text columns and drop everything except the 'content' and 'label' columns

In [13]:
#Combine title and text into 'content' column
data['content'] = data['title'] + ' ' + data['text']

#Drop other columns (except for label)
data = data.drop(columns=['title', 'text', 'subject', 'date'])

#Check data
data

,label,content
0,0,Donald Trump Sends Out Embarrassing New Year’...
1,0,Drunk Bragging Trump Staffer Started Russian ...
2,0,Sheriff David Clarke Becomes An Internet Joke...
3,0,Trump Is So Obsessed He Even Has Obama’s Name...
4,0,Pope Francis Just Called Out Donald Trump Dur...
...,...,...
44893,1,'Fully committed' NATO backs new U.S. approach...
44894,1,LexisNexis withdrew two products from Chinese ...
44895,1,Minsk cultural hub becomes haven from authorit...
44896,1,Vatican upbeat on possibility of Pope Francis ...


## Step 1c: Preprocessing

Next, we will do some simple cleaning of the 'content' column:
- Convert it to lowercase
- Remove punctuation
- Remove stopwords

In [25]:
#Create a function for the data preprocessing tasks
def preprocessing(text):
    #Turn to lowercase
    text = text.lower()

    #Remove punctuation 
    text = re.sub(r'[^\w\s]', '', text)

    #Remove stopwords
    #Split text into individual words
    split_text = text.split()
    cleaned_text = []
    
    for word in split_text:
        if word not in stop_words:
            #Only keep non-stop word words
            cleaned_text.append(word)

    #Recombine into one string
    rejoined_text = ' '.join(cleaned_text)

    return rejoined_text

In [27]:
#Apply preprocessing function to data
data['content'] = data['content'].apply(preprocessing)

#Check to make sure it looks right
data

,label,content
0,0,donald trump sends embarrassing new years eve ...
1,0,drunk bragging trump staffer started russian c...
2,0,sheriff david clarke becomes internet joke thr...
3,0,trump obsessed even obamas name coded website ...
4,0,pope francis called donald trump christmas spe...
...,...,...
44893,1,fully committed nato backs new us approach afg...
44894,1,lexisnexis withdrew two products chinese marke...
44895,1,minsk cultural hub becomes authorities minsk r...
44896,1,vatican upbeat possibility pope francis visiti...


## Step 1d: Train/Validation/Test Splitting

Now, we will split our cleaned data into training, validation, and testing sets. This will be useful for tuning hyperparameters and assessing model performance later on. 

The split we will choose is: 
- **Train**: 70%
- **Validation**: 15%
- **Test**: 15%

In [31]:
#Split into features (X) and labels (y)
X = data['content']
y = data['label']

In [ ]:
# Split into train and temp 
train, temp_data = train_test_split(data, test_size=0.3, stratify=data['label'], random_state=42)

# Split temp into validation & test
val, test = train_test_split(temp_data, test_size=0.5, stratify=temp_data['label'], random_state=42)

# Get features and labels for each split
X_train, y_train = train['content'], train['label']
X_val, y_val = val['content'], val['label']
X_test, y_test = test['content'], test['label']

In [ ]:
# Display the distribution of Fake and Real News in nonprocessed and processed datasets after splitting

print("Processed Data Train, Validation, & Test Distribution")
print("Train Labels:", y_train.value_counts(normalize=True))
print("Validation Labels:", y_val.value_counts(normalize=True))
print("Test Labels:", y_test.value_counts(normalize=True))

## Step 1e: Apply TF-IDF Vectorization

**TF-IDF** is a combination of 2 terms:
- **Term Frequency (TF)**: how frequently a term appears in one given document ($d$), calculated as:
$$
\text{TF(term, document)} = \frac{\text{Number of times the term appears in the document}}{\text{Total number of terms in the document}}
$$

- **Inverse Document Frequency (IDF)**: how unique the term ($t$) is across all the documents ($d$), calculated as:
$$
\text{IDF(term, corpus)} = \log \left( \frac{\text{Total number of documents}}{\text{1 + Number of documents containing the term}} \right)
$$

In practice, TF-IDF is a weighting technique that **gives higher weight to**:
- **High TF score** (frequent in one document)
- and **Low IDF score** (not too common across all documents)

Specifically, we will apply a **Unigram** TF-IDF vectorization method. This means that we will use **single words** (no phrases or pairs of words).

**We will fit the vectorizer on the training data only** to avoid 'cheating' (aka allowing the model to learn from the data in the validation/testing sets, which would give it an unfair advantage in later testing). We will then transform all the splits.

This will result in sparse matrices that transform the words of the text into numerical values suitable for the SVM to interpret.

In [46]:
#Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(
    ngram_range=(1, 1),       #Unigrams
    stop_words='english'      #Remove stop words again (just in case)
)

In [48]:
#Fit vectorizer on training data ONLY and transform
X_train_tfidf = vectorizer.fit_transform(X_train)

#Transform (NO FITTING TO AVOID CHEATING) validation, test sets
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

Count Vectorizer converts text data into numerical features. It captures word usage patterns in the articles by representing each article as a vector of word frequencies.

In [ ]:
# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_counts = vectorizer.fit_transform(X_train)

# Transform the validation and test data
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

# Step 2: Implementing Logistic Regression, Random Forest, Naive Bayes, & SVM

For each model, two versions were evaluated: 1 using TF-IDF and the other using Count Vectorizer. The precision, recall, and F1 score was calculated using the predictions and test set. A confusion matrix and classification report was printed for evaluation on both test and validation sets. 

In [ ]:
# Create function for calculating and printing precision, recall, f1 score, & confusion matrix 
# (only called for test set)
def model_evaluation(test, pred, name:string):
    precision = precision_score(test, pred)
    recall = recall_score(test, pred)
    f1 = f1_score(test, pred)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    print(f"\n{name} Classification Report:")
    print(classification_report(test, pred))

    cm = confusion_matrix(test, pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
    plt.xlabel(f'Predicted Labels ({name})')
    plt.ylabel('True Labels')
    plt.title(f'{name} Confusion Matrix')
    plt.show()

**Logistic Regression**

Logistic Regression is a linear model that calculates the probability of a binary problem (fake or real) based on a linear combo of input features (word counts/frequencies from TF-IDF & Count Vectorizer). It uses a sigmoid function to map the linear combo to a probability between 0 and 1 (0 is fake and 1 is real). 

In [ ]:
# Initialize & train Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)  
logistic_model.fit(X_train_tfidf, y_train)

# Make predictions on test set
lr_y_pred = logistic_model.predict(X_test_tfidf)

# Evaluate model
model_evaluation(y_test, lr_y_pred, "Logistic Regression")

# Validation set evaluation
y_val_pred = logistic_model.predict(X_val_tfidf)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

In [ ]:
# With Count Vectorizer
logistic_model = LogisticRegression(max_iter=1000, random_state=42)  
logistic_model.fit(X_train_counts, y_train)

lr_y_pred = logistic_model.predict(X_test_counts)

model_evaluation(y_test, lr_y_pred, "Logistic Regression")

# Validation set evaluation
y_val_pred = logistic_model.predict(X_val_counts)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

**Random Forest**

Random Forest builds multiple decision trees and combines their predictions. Each decision tree learns to classify articles as real or fake based on word frequencies. 

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_tfidf, y_train)
rf_y_pred = rf_model.predict(X_test_tfidf)

model_evaluation(y_test, rf_y_pred, "Random Forest")

# Validation set evaluation
y_val_pred = rf_model.predict(X_val_tfidf)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

In [ ]:
# With Count Vectorizer
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_counts, y_train)
rf_y_pred = rf_model.predict(X_test_counts)

model_evaluation(y_test, rf_y_pred, "Random Forest")

# Validation set evaluation
y_val_pred = rf_model.predict(X_val_counts)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

**Naive Bayes**

Naive Bayes (MultinomialNB) is a probabilistic model that calculates the probability of an article being real or fake based on the word frequencies in the article. It assumes the features (word counts) are independent from one another (essentially the naive assumption). It then uses Bayes Theorem to calculate the conditional probabilities. 

In [ ]:
# MultinomialNB works with TF-IDF data
nb_model = MultinomialNB()  
nb_model.fit(X_train_tfidf, y_train)
nb_y_pred = nb_model.predict(X_test_tfidf)

model_evaluation(y_test, nb_y_pred, "Naive Bayes")

# Validation set evaluation
y_val_pred = nb_model.predict(X_val_tfidf)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

In [ ]:
# With Count Vectorizer
nb_model = MultinomialNB()  
nb_model.fit(X_train_counts, y_train)
nb_y_pred = nb_model.predict(X_test_counts)

model_evaluation(y_test, nb_y_pred, "Naive Bayes")

# Validation set evaluation
y_val_pred = nb_model.predict(X_val_counts)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

**SVM**

In [ ]:
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train_tfidf, y_train)

# Predictions on the test set
y_pred = svm_model.predict(X_test_tfidf)

model_evaluation(y_test, y_pred, "SVM")

# Validation set evaluation
y_val_pred = svm_model.predict(X_val_tfidf)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

In [ ]:
# With Count Vectorizer
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train_counts, y_train)

# Predictions on the test set
y_pred = svm_model.predict(X_test_counts)

model_evaluation(y_test, y_pred, "SVM")

# Validation set evaluation
y_val_pred = svm_model.predict(X_val_counts)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

cm_val = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

**Hyperparameter Tuning on SVM Model**

The C variable in SVM models is the regularization parameter which controls the level of trade-off between maximizing the margin and minimizing classification error. A high C results in the model trying to classify each training sample correctly which can cause a smaller margin (less tolerance for misclassified points) but can lead to overfitting. A low C allows some misclassifications to get a larger margin and can lead to generalization of data (sometimes underfitting the data). 

In [ ]:
# Using count vectorizer since it yielded a higher F1 for SVM compared to TF-IDF

C_vals = [0.01, 0.1, 1, 10, 100]

best_f1 = 0
best_C = None

for C in C_vals:
    svm_model = SVC(kernel='linear', C=C, random_state=42)
    svm_model.fit(X_train_counts, y_train) 
    y_val_pred = svm_model.predict(X_val_counts)
    report = classification_report(y_val, y_val_pred, output_dict=True)

    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1 = report['weighted avg']['f1-score']

    print(f"C: {C}, Precision: {precision:.10f}, Recall: {recall:.10f}, F1: {f1:.10f}")

    if f1 > best_f1:
        best_f1 = f1
        best_C = C

print(f"Best C: {best_C}, Best Validation F1: {best_f1}")

# Train final model (with best c) on combined training & validation sets
final_svm = SVC(kernel='linear', C=best_C, random_state=42)
final_svm.fit(X_train_counts, y_train)
y_test_pred = final_svm.predict(X_test_counts)
print(classification_report(y_test, y_test_pred))

In [ ]:
cm = confusion_matrix(y_val, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()